In [ ]:
import re
import os
import sys
import csv
import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

from IPython.display import display, HTML

In [ ]:
INPUT_DATA_PATH = '../data/'
CLEANED_DATA_PATH = '../data/cleaned/'
if not os.path.exists(CLEANED_DATA_PATH): os.makedirs(CLEANED_DATA_PATH)

### IMPORTANT
Function for reading CSV with non-escaped delimiters in *Terms* column

In [ ]:
def bad_csv_line_consultations(line):
    assert len(line[0]) == 64 # confirm pseudo ID in first column
    assert line[1].isnumeric # confirm patient ID in second column
    assert line[2].isnumeric # confirm consultation ID in third column
    try: pd.to_datetime(line[3]) # confirm for date in fourth column
    except: raise ValueError('This should be a date', line)
    assert line[4].isalpha # confirm consultation type in fifth column
    code_column = 6
    while not re.search(r'^(?=.*[0-9])(?=.*[A-Z])([A-Z0-9]+)$', line[code_column]): code_column += 1 # check for GP code
    return line[:5] + [','.join(line[5:code_column])] + line[code_column:] # merge Term columns

Extraction end dates:
- **LDN** - September 2021 (most recent LDN update in CRIS)
- **SLaM** - June 2022

**NOTE** CSV Python engine seems to put read details in last two rows

In [ ]:
ldn_consultations_load = pd.read_csv(INPUT_DATA_PATH + 'output2.csv', engine='python', on_bad_lines=bad_csv_line_consultations, dtype=str)
assert ldn_consultations_load.iloc[-2, :].str.contains(r'^\([0-9]* rows affected\)$', regex=True).iloc[0]
assert ldn_consultations_load.iloc[-1, :].str.contains(r'^Completion time: [0-9T:\-\+\.]*$', regex=True).iloc[0]
ldn_consultations_load = ldn_consultations_load[:-2]

In [ ]:
slam_contacts_load = pd.read_excel(INPUT_DATA_PATH + 'slam_outputs_1_2.xlsx', sheet_name='output1_events', dtype=str)

In [ ]:
slam_admissions_load = pd.read_excel(INPUT_DATA_PATH + 'slam_outputs_1_2.xlsx', sheet_name='output2_admissions', dtype=str)

In [ ]:
patient_level_load = pd.read_excel(INPUT_DATA_PATH + 'ldn_outputs1_3.xlsx', sheet_name='output1', dtype=str)

In [ ]:
ldn_medications_load = pd.read_excel(INPUT_DATA_PATH + 'ldn_outputs1_3.xlsx', sheet_name='output3', dtype=str)

## Cleaning
- Parsing and initial cleaning of datasets
- Removing those duplicate registration rows

### Functions

In [ ]:
def check_id_format_and_standardise(df, patient_id_column):
    ### confirm that the patient identifiers are parsed as strings
    ### and confirm that pseudo-NHS numbers are exactly 64 characters
    ### and patient IDs are less than or equal to 7 characters
    df = df.copy()
    df = df.rename(columns={'PseudonymisedNHSNumber':'nhs', patient_id_column:'patient_id'})
    ### assertions
    assert df.nhs.notna().all()
    assert df.nhs.apply(lambda x: isinstance(x, str)).all()
    assert df.nhs.apply(len).eq(64).all()
    assert df.patient_id.notna().all()
    assert df.patient_id.apply(lambda x: isinstance(x, str)).all()
    assert df.patient_id.apply(len).le(7).all()
    ### assert that ID-to-NHS is 1-to-1
    assert df.groupby('patient_id').nhs.nunique().eq(1).all()
    ### then zero-pad patient IDs to 7 characters
    df.patient_id = df.patient_id.str.zfill(7)
    ### return checked and padded dataframe
    return df

def filter_to_inclusion_cohort(df, patient_level_nhs):
    ### all LDN  outputs are for active LDN registration from 2019 AND inclusion of LTC
    ### all SLAM outputs are for active LDN registration from 2019 ONLY
    ### i.e. SLAM outputs could also include LDN patients with any conditions and without our LTCs!
    df = df.copy()
    patient_level_nhs = patient_level_nhs.copy()
    ### remove irrelevant patients from SLAM outputs who do not meet our LTC inclusion criteria (theoretically)
    df = df.loc[df.nhs.isin(patient_level_nhs)]
    ### return filtered dataframe
    return df


def get_id_counts_and_sets(df, patient_level_id_pairs):
    ### but some patient IDs from SLAM outputs will not appear in LDN patient level EVEN if that patient NHS number is in our patient level
    ### this is because the patient ID pertains to a GP registration, and our LTCs maybe not appear during that specific registration
    ### combine dataframe IDs with patient level, and then get count lookup table
    df_and_patient_level_ids = pd.concat([df, patient_level_id_pairs])[['nhs', 'patient_id']].copy()
    patient_id_counts = df_and_patient_level_ids.groupby('nhs').patient_id.nunique().to_dict()
    ### all pseudo-NHS numbers, and all patient IDs in cohort
    pseudo_nhs_all = set(df_and_patient_level_ids.nhs)
    patient_ids_all = set(df_and_patient_level_ids.patient_id)
    ### return lookup table, and sets of pseudo-NHS and IDs
    return patient_id_counts, pseudo_nhs_all, patient_ids_all

    
def check_patient_ids(patient_ids):
    ### checks the patient IDs for SEEMINGLY duplicate patient events
    ### checks everything is the same, including pseudo-NHS number, EXCEPT the patient IDs
    ### so pseudo-NHS is the same in each group (grouped by this too)
    ### only patient ID is being checked, and could possibly be different values

    ### in theory, should only be one patient ID as grouping by pseudo-NHS number!
    ### but some patients have multiple patient IDs, one per GP registration,
    ### and this seems to have resulted in duplicate events, POTENTIALLY one per unique patient ID

    ### issue is it's possible for a patient to have two events i.e. separate events on the same day, but look the same for each data-point!
    ### so we need to de-duplicate the actual duplicates, and leave behind the unique event and any similar but separate events!
    
    ### patient_ids is just a list of the all the patient IDs
    ### patient IDs are not null

    num_events = len(patient_ids)
    unique_ids = patient_ids.unique()
    num_unique_ids = patient_ids.nunique()
    ids_value_counts = patient_ids.value_counts()
    
    nhs_in_patient_level = patient_ids.name[0] in pseudo_nhs_all
    id_in_patient_level = all([p_id in patient_ids_all for p_id in unique_ids])
    nhs_in_patient_level_tick = '✔' if nhs_in_patient_level else '✘'
    id_in_patient_level_tick = '✔' if id_in_patient_level else '✘'

    # print('---')
    # print(f"Events: {num_events}", ids_value_counts.to_dict())

    mismatch_msg = ''
    
    if nhs_in_patient_level and id_in_patient_level:
        # print('NHS number matched ☑ | Patient ID (registration) matched ☑')
        num_recorded_duplicate_ids = patient_id_counts[patient_ids.name[0]]
        # make sure the number of IDs per NHS (on record) matches the number of IDs for this event
        # assert num_recorded_duplicate_ids == num_unique_ids, patient_ids
        if num_recorded_duplicate_ids != num_unique_ids:
            msg = f'Number of IDs on record ({num_recorded_duplicate_ids}) do not match number of IDs for this event ({num_unique_ids})'
            # display(HTML(f'<p style="background-color: red;">{msg}</p>'))
            mismatch_msg = ' MM'
    else:
        msg = f'NHS number matched {nhs_in_patient_level_tick} | Patient ID (registration) matched {id_in_patient_level_tick}'
        # display(HTML(f'<p style="background-color: red;">{msg}</p>'))
        return 'Missing ID'  + mismatch_msg
    
    if num_events == 1:
        msg = f'Single event for single patient and matching NHS number associated patient IDs ({num_unique_ids}) ☑'
        # display(HTML(f'<p style="background-color: lightgreen;">{msg}</p>'))
        return f'Single ({num_unique_ids})' + mismatch_msg
    elif num_unique_ids == 1:
        msg = f'Similar events only ({num_events} events) ☑'
        # display(HTML(f'<p style="background-color: lightblue;">{msg}</p>'))
        return f'Similar ({num_events})' + mismatch_msg
    elif ids_value_counts.eq(1).all():
        msg = f'Possible duplicates patient IDs ({num_unique_ids}) and matching NHS number associated patient IDs ({num_recorded_duplicate_ids}) ☑'
        # display(HTML(f'<p style="background-color: yellow;">{msg}</p>'))
        return f'Duplicate ({num_unique_ids})' + mismatch_msg
    elif ids_value_counts.nunique() == 1:
        assert num_events % num_unique_ids == 0
        msg = f'Possible duplicates ({unique_ids} patient IDs) AND similar events ({num_events // num_unique_ids} events) ☑'
        # display(HTML(f'<p style="background-color: plum;">{msg}</p>'))
        return f'Similar and Duplicate ({num_events // num_unique_ids}, {num_unique_ids})' + mismatch_msg
    else:
        raise('Something is wrong...')


def assign_duplicate_type(df, patient_level_id_pairs):
    # make a copy of the input dataframe
    df = df.copy()
    patient_level_id_pairs = patient_level_id_pairs.copy()
    # check all NHS match patient level
    assert df.nhs.isin(patient_level_id_pairs.nhs).eq(True).all()
    print(f"All NHS numbers matched to patient level. Patient IDs unmatched: {(~df.patient_id.isin(patient_level_id_pairs.patient_id)).sum()}/{df.shape[0]}\n")
    # apply group checking function
    df['Status'] = df.groupby(list(df.columns.drop('patient_id')), dropna=False).patient_id.progress_transform(check_patient_ids)
    sys.stdout.flush()  # <-- Add this to flush the buffer
    sys.stderr.flush()  # <-- And this for stderr
    print('\nGroup assignment complete!\n')
    display(df.Status.value_counts().sort_index())
    # return dataframe
    return df
    

def de_duplicate_group(patient_ids):
    # parse status
    status_string = patient_ids.name[-1]
    status, other = status_string.strip(')').split(' (')
    assert status in ['Single', 'Similar', 'Duplicate', 'Similar and Duplicate']

    num_events = len(patient_ids)
    num_unique_ids = patient_ids.nunique()

    if status == 'Single':
        assert num_events == 1
        num_keep = 1
    elif status == 'Similar':
        num_events_parsed = int(other)
        assert num_unique_ids == 1
        assert num_events == num_events_parsed
        num_keep = num_events
    elif status == 'Duplicate':
        num_unique_ids_parsed = int(other)
        assert num_unique_ids == num_unique_ids_parsed
        assert num_events == num_unique_ids_parsed
        num_keep = 1
    elif status == 'Similar and Duplicate':
        num_events_parsed, num_unique_ids_parsed = (int(x) for x in other.split(','))
        assert num_unique_ids == num_unique_ids_parsed
        assert num_events % num_unique_ids == 0
        assert (num_events // num_unique_ids) == num_events_parsed
        num_keep = num_events_parsed
    else:
        raise('Something is wrong...')

    # Return 0, 1, 2, ... num_keep-1, then NaN for rest
    assert isinstance(num_keep, int)
    return pd.Series(
        list(range(num_keep)) + [np.nan] * (len(patient_ids) - num_keep),
        index=patient_ids.index
    )

def de_duplicate_dataframe(df):
    # make a copy of the input dataframe
    df = df.copy()
    # apply de-duplication function, and filter out duplicate rows
    print('\nDe-duplicating dataframe...\n')
    df['EVENT_ID'] = df.groupby(list(df.columns.drop(['patient_id'])), dropna=False).patient_id.progress_transform(de_duplicate_group)
    df_de_duplicated = df.loc[df.EVENT_ID.notna()].copy()
    # format dataframe
    df_de_duplicated.EVENT_ID = df_de_duplicated.EVENT_ID.astype(int)
    df_de_duplicated = df_de_duplicated.drop(['patient_id', 'Status'], axis=1)
    sys.stdout.flush()  # <-- Add this to flush the buffer
    sys.stderr.flush()  # <-- And this for stderr
    print(f"\nChecks complete and de-duplication finished!")
    print(f"Previous size: {df.shape[0]} | Resulting size: {df_de_duplicated.shape[0]} | Removed {df.shape[0] - df_de_duplicated.shape[0]} duplicated events")
    # return de-depulicated dataframe
    return df_de_duplicated
    # return df

### Patient Level

In [ ]:
# copy from loading dataframe
patient_level = patient_level_load.copy()

In [ ]:
# format date columns
patient_level.yearofbirth = pd.to_datetime(patient_level.yearofbirth, format='mixed').dt.normalize()
patient_level.dateofdeath = pd.to_datetime(patient_level.dateofdeath, format='mixed').dt.normalize()
patient_level.registrationstartdate = pd.to_datetime(patient_level.registrationstartdate, format='mixed').dt.normalize()
patient_level.RegistrationEndDate = pd.to_datetime(patient_level.RegistrationEndDate, format='mixed').dt.normalize()

# format boolean columns
patient_level.SLAM_SMI_ever = patient_level.SLAM_SMI_ever.map({'0': False, '1': True})
patient_level.SLAM_depression_ever = patient_level.SLAM_depression_ever.map({'0': False, '1': True})
patient_level.SLAM_anxiety_ever = patient_level.SLAM_anxiety_ever.map({'0': False, '1': True})

# format integer columns
patient_level.LDN_N_anxiety_ever = patient_level.LDN_N_anxiety_ever.astype(int)
patient_level.LDN_N_depression_ever = patient_level.LDN_N_depression_ever.astype(int)
patient_level.LDN_N_SMI_ever = patient_level.LDN_N_SMI_ever.astype(int)
patient_level.LDN_N_AF_ever = patient_level.LDN_N_AF_ever.astype(int)
patient_level.LDN_N_heart_failure_ever = patient_level.LDN_N_heart_failure_ever.astype(int)
patient_level.LDN_N_IHD_ever = patient_level.LDN_N_IHD_ever.astype(int)
patient_level['Index of Multiple Deprivation (IMD) Rank (where 1 is most depriv'] = patient_level['Index of Multiple Deprivation (IMD) Rank (where 1 is most depriv'].astype('Int32')

In [ ]:
# rename and standardise ID columns
patient_level = check_id_format_and_standardise(patient_level, 'patientid')

Add IMD deciles

In [ ]:
patient_level = patient_level.rename(columns={'Index of Multiple Deprivation (IMD) Rank (where 1 is most depriv':'IMD_Rank', 'sex_LDN':'Sex'})
patient_level['IMD_Decile'] = (1 + np.floor(10 * (patient_level['IMD_Rank'] - 1) / 32_844)).astype('Int32')

Add ethnicity categories

In [ ]:
### QUESTIONABLE (MANUAL) ASSIGNMENT OF CENSUS 2021 CATEGORIES
### SHOULD USE OFFICAL READ CODE TO SNOMED, AND SNOMED TO CENSUS 2021 LOOKUP/MAPPING
ethnicities = pd.read_excel('./Resources/Ethnicities.xlsx', usecols=['description', 'A/AB', 'B/BB/C/A', 'M/ME', 'W', 'O', 'UNK'])

In [ ]:
ethnicity_categories = {
    'A/AB':'Asian or Asian British',
    'B/BB/C/A':'Black, Black British, Caribbean or African',
    'M/ME':'Mixed or multiple ethnic groups',
    'W':'White',
    'O':'Other ethnic group',
    'UNK':'Missing'
}

In [ ]:
ethnicities['Ethnicity'] = ethnicities.iloc[:, 1:].idxmax(axis=1)
ethnicities['Ethnicity'] = ethnicities['Ethnicity'].replace(ethnicity_categories)
ethnicities = ethnicities.rename(columns={'description':'LDN_ethnicity'})
ethnicities = ethnicities.drop_duplicates()

# merge in ethnicities
patient_level = patient_level.merge(ethnicities[['LDN_ethnicity', 'Ethnicity']], on='LDN_ethnicity', how='left')

# change Missing to NaN
patient_level.Ethnicity = patient_level.Ethnicity.replace({'Missing':np.nan})

Language

In [ ]:
# First_Language_ID - this is mainly just ensure a valid value is propogated to the top, if it exists
patient_level['First_Language_ID'] = patient_level['First_Language_ID'].replace({'Not Known':np.nan, 'Other':np.nan})

# LDN_preferred_language
### HAVEN'T HANDLED THIS YET - NOT USING CURRENTLY...

Create lists for ID mappings and dynamic values

In [ ]:
# save NHS-ID pairs for de-duplication of other outputs
patient_level_id_pairs = patient_level[['patient_id', 'nhs']].drop_duplicates().copy()

# save NHS to GP details
patient_level_gp_lookup = patient_level[['nhs', 'IMD_Decile', 'NationalPracticeCode', 'registrationstartdate', 'RegistrationEndDate']].drop_duplicates().copy()

# save dynamic values to file - IMD and GP
patient_level_gp_lookup.to_csv(CLEANED_DATA_PATH + 'Patient_Level_Dynamic.csv', index=False, quoting=csv.QUOTE_ALL)

Reconcile demographic data

In [ ]:
# 1. Sort by most recent registration (Crucial for 'first' logic)
patient_level = patient_level.sort_values('registrationstartdate', ascending=False)

# 2. PREPARATION: Handle NaNs for sum columns only
# We fill NaNs with 0 here so the sum works correctly. 
# We do NOT backfill these.
sum_columns = ['LDN_N_anxiety_ever', 'LDN_N_depression_ever', 'LDN_N_SMI_ever',  'LDN_N_AF_ever', 'LDN_N_heart_failure_ever', 'LDN_N_IHD_ever']

patient_level[sum_columns] = patient_level[sum_columns].fillna(0)

# 3. BUILD DICTIONARY
# We define the logic: 'sum' for counts, 'first' for everything else.
# Optimization Note: In pandas groupby, 'first' skips NaNs automatically.
agg_dict = {
    col: 'sum' if col in sum_columns else 'first' 
    for col in patient_level.columns 
    if col != 'nhs' and col != 'patient_id'
}

# 4. AGGREGATE (The Speed Optimization)
# Instead of backfilling first (slow), we rely on .agg()
patient_level = patient_level.groupby('nhs', as_index=False).agg(agg_dict)

# 5. CLEANUP
# patient_id is no longer needed (nhs is the grouper)
if 'patient_id' in patient_level.columns: patient_level = patient_level.drop('patient_id', axis=1)

In [ ]:
print('Number of unique patients:', patient_level.shape[0])
patient_level.sample(2).T

In [ ]:
# save to file
assert not patient_level.duplicated().any()
assert not patient_level.nhs.duplicated().any()
patient_level.to_csv(CLEANED_DATA_PATH + 'Patient_Level.csv', index=False) # save newly cleaned version to CSV

### SLAM Admissions

In [ ]:
# copy from loading dataframe
slam_admissions = slam_admissions_load.copy()

In [ ]:
# format date columns
slam_admissions.Admission_Date = pd.to_datetime(slam_admissions.Admission_Date, format='mixed').dt.normalize()
slam_admissions.Discharge_Date = pd.to_datetime(slam_admissions.Discharge_Date, format='mixed').dt.normalize()

In [ ]:
# rename and standardise ID columns
slam_admissions = check_id_format_and_standardise(slam_admissions, 'PatientId')

In [ ]:
# get counts and sets of all NHS and IDs
patient_id_counts, pseudo_nhs_all, patient_ids_all = get_id_counts_and_sets(slam_admissions, patient_level_id_pairs)

# filter to cohort and de-duplicate patient IDs
slam_admissions = filter_to_inclusion_cohort(slam_admissions, patient_level_id_pairs.nhs)
slam_admissions = assign_duplicate_type(slam_admissions, patient_level_id_pairs)
slam_admissions = de_duplicate_dataframe(slam_admissions)

In [ ]:
# only groups with SIMILAR statuses (multiple valid events) will result in multiple events, and therefore with some EVENT_ID > 0
slam_admissions.EVENT_ID.value_counts(dropna=False)

In [ ]:
print('Number of unique SLAM admissions:', slam_admissions.shape[0])
slam_admissions.sample(10)

In [ ]:
# save to file
assert slam_admissions.Admission_Date.notna().all()
slam_admissions.to_csv(CLEANED_DATA_PATH + 'SLaM_Admissions.csv', index=False) # save newly cleaned version to CSV

### SLAM Contacts

Replace modalities with standard categories (as per LDN):
- F2F
- Telephone
- Home Visit
- Video/Email/Text
- Other
- Missing

In [ ]:
# copy from loading dataframe
slam_contacts = slam_contacts_load.copy()

In [ ]:
# format date columns
slam_contacts.SLAM_event_date = pd.to_datetime(slam_contacts.SLAM_event_date, format='mixed').dt.normalize()

In [ ]:
# define modality lookup table
modality = {
 'ECT Treatment': 'F2F',
 'Face To Face': 'F2F',
 'Face to Face': 'F2F',
 'Group Contact': 'F2F',
 'Group Contact (Contract Option)': 'F2F',
 'Individual Contact (Contract Option)': 'F2F',
 'Observation': 'F2F',
 'Phone - Clinical': 'Telephone',
 'Phone - Triage': 'Telephone',
 'RC discussion after SOAD': 'F2F',
 'Statutory Consultee after SOAD': 'F2F',
 'Video (virtual) appointment': 'Video',
 'Video (virtual) appointment (Contract Option)': 'Video',
 'Video link': 'Video',
 'f2f': 'F2F',
 'na': 'Missing'
}

# parse and format modality
slam_contacts['Modality'] = slam_contacts['event_type_of_contact'].replace(modality)
slam_contacts['Modality'] = slam_contacts['Modality'].where((slam_contacts['Modality'] != 'Missing') | (slam_contacts['dimension_1_medium'] != 'f2f'), other='F2F')
slam_contacts['Modality'].value_counts(dropna=False)

In [ ]:
# rename and standardise ID columns
slam_contacts = check_id_format_and_standardise(slam_contacts, 'PatientId')

In [ ]:
# get counts and sets of all NHS and IDs
patient_id_counts, pseudo_nhs_all, patient_ids_all = get_id_counts_and_sets(slam_contacts, patient_level_id_pairs)

# filter to cohort and de-duplicate patient IDs
slam_contacts = filter_to_inclusion_cohort(slam_contacts, patient_level_id_pairs.nhs)
slam_contacts = assign_duplicate_type(slam_contacts, patient_level_id_pairs)
slam_contacts = de_duplicate_dataframe(slam_contacts)

In [ ]:
print('Number of unique SLAM contacts:', slam_contacts.shape[0])
slam_contacts.sample(2).T

In [ ]:
# save to file
assert slam_contacts.SLAM_event_date.notna().all()
slam_contacts.to_csv(CLEANED_DATA_PATH + 'SLaM_Contacts.csv', index=False) # save newly cleaned version to CSV

### LDN Consultations

In [ ]:
# copy from loading dataframe
ldn_consultations = ldn_consultations_load.copy()

In [ ]:
# format date columns (and filter to good dates)
ldn_consultations.EffectiveDateTime = pd.to_datetime(ldn_consultations.EffectiveDateTime, format='mixed', errors='coerce')
assert ldn_consultations.EffectiveDateTime.isna().sum() == 1 # should only be only bad date
ldn_consultations = ldn_consultations.loc[ldn_consultations.EffectiveDateTime.notna()]

In [ ]:
# rename and standardise ID columns
ldn_consultations = check_id_format_and_standardise(ldn_consultations, 'PatientId')

In [ ]:
# replace HCP with high-level categories
ldn_hcp_lookup = pd.read_excel('./Resources/LDN HCP Lookup.xlsx')
ldn_hcp_lookup['TypeHCP'] = ldn_hcp_lookup.iloc[:, -1].replace({1:'GP', 2:'Nurse or Midwife', 3:'Other HCP or Unspecified', 4:'Admin'})
ldn_hcp_lookup_dict = {hcp_name:hcp_type for hcp_name, hcp_type in zip(ldn_hcp_lookup['UserRoleName'], ldn_hcp_lookup['TypeHCP'])}

# rename users
ldn_consultations['userrolename'] = ldn_consultations['userrolename'].replace(ldn_hcp_lookup_dict)
ldn_consultations['userrolename'].value_counts(dropna=False)

Consultation type: 1 = GP; 2 = nurse or midwife; 3 = other HCP or unspecified clinical staff; 4 = admin

In [ ]:
# replace modality with high-level categories
ldn_modality_lookup = pd.read_excel('./Resources/LDN Modality Lookup.xlsx')
ldn_modality_lookup['Modality'] = ldn_modality_lookup.iloc[:, 2].replace({1:'F2F', 2:'Telephone', 3:'Home Visit', 4:'Video/Email/Text', 5:'Other', np.nan:'Other'})
ldn_modality_lookup_dict = {mod_name:mod_type for mod_name, mod_type in zip(ldn_modality_lookup['top_ConsultationTypeDescription'], ldn_modality_lookup['Modality'])}

Add in other modalities present in LDN extraction, but not in lookup

**NOTE** Hand-picked lookup values

In [ ]:
other_modality = {
 'Consultation via SMS text message': 'Video/Email/Text',
 'Home visit note': 'Other',
 'Third party consultation': 'Other',
 'Face to face consultation': 'F2F',
 'Patient encounter data NOS': 'Other',
 'Repeat prescription': 'Other',
 'Consultation via telemedicine web camera': 'Video/Email/Text',
 'Night visit note': 'Other',
 'Emergency appointment': 'Other',
 'Consultation via multimedia': 'Video/Email/Text',
 'EMIS patient record transfer': 'Other',
 'Extended hours consultation': 'Other',
 'Discussion with other professional': 'Other',
 'OOH report': 'Other',
 'Diagnostic testing': 'Other',
 'Same day appointment': 'Other',
 'Follow up attendance face to face': 'F2F',
 'First attendance face to face': 'F2F',
 'Seen in other clinic': 'Other',
 'Community care activity type': 'Other',
 'Hospital inpatient note': 'Other',
 'Appt cancelled by patient': 'Other',
 'Walk-in centre': 'Other',
 'Genito-urinary medicine': 'Other',
 'Multidisciplinary team meeting without patient': 'Other'}

In [ ]:
# update modality lookup with manual values
ldn_modality_lookup_dict.update(other_modality)

### KEEP GRANULAR MODALITIES
ldn_consultations['GranularModality'] = ldn_consultations['ConsultationTypeTerm']

# replace modality names
ldn_consultations['ConsultationTypeTerm'] = ldn_consultations['ConsultationTypeTerm'].replace(ldn_modality_lookup_dict)
assert all(ldn_consultations['ConsultationTypeTerm'].dropna().isin(ldn_modality_lookup['Modality'].unique())) # all assigned category

# replace nulls with Missing category
ldn_consultations['ConsultationTypeTerm'] = ldn_consultations['ConsultationTypeTerm'].replace({np.nan:'Missing'})

**NOTE** Choosing to merge home visits into F2F, and *Other* into *Missing*

**UPDATE** *Other* into **np.nan**

In [ ]:
# ldn_consultations['ConsultationTypeTerm'] = ldn_consultations['ConsultationTypeTerm'].replace({'Home Visit':'F2F', 'Other':'Missing'})

ldn_consultations['ConsultationTypeTerm'] = ldn_consultations['ConsultationTypeTerm'].replace({'Home Visit':'F2F'})
ldn_consultations = ldn_consultations.loc[ldn_consultations.ConsultationTypeTerm != 'Other'] # drop admin (other) consults

ldn_consultations['ConsultationTypeTerm'].value_counts(dropna=False)

In [ ]:
valid_consultation_fields = ['nhs', 'EffectiveDateTime', 'ConsultationTypeTerm', 'userrolename', 'GranularModality']

# ignore patient ID, consultation ID, GP name, GP code, and just get unique consultations (based mainly on time and modality)
ldn_consultations = ldn_consultations[valid_consultation_fields].drop_duplicates()

### then for that patient (NHS) and that date, find the most recently registered GP practice for them which overlaps this date

# Merge on NHS number
ldn_consultations = ldn_consultations.merge(patient_level_gp_lookup, on='nhs', how='left')

# Create mask of registrations which overlap with consultation
mask = (
    (ldn_consultations['registrationstartdate'] <= ldn_consultations['EffectiveDateTime']) &
    ((ldn_consultations['RegistrationEndDate'] >= ldn_consultations['EffectiveDateTime']) | (ldn_consultations['RegistrationEndDate'].isna()))
)

# Filter to registrations that overlap with consultation date
ldn_consultations = ldn_consultations[mask].copy()

# For each consultation, keep only the most recent registration (by registrationstartdate)
ldn_consultations = ldn_consultations.sort_values('registrationstartdate', ascending=False)
ldn_consultations = ldn_consultations.groupby(valid_consultation_fields, as_index=False, dropna=False).first()

# Drop registration dates (just keep GP code)
ldn_consultations = ldn_consultations.drop(['registrationstartdate', 'RegistrationEndDate'], axis=1)

In [ ]:
# save to file
assert not ldn_consultations.duplicated().any()
ldn_consultations.to_csv(CLEANED_DATA_PATH + 'LDN_Consultations.csv', index=False, quoting=csv.QUOTE_ALL) # save newly cleaned version to CSV